In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=32156 sha256=576b8c452da0e1f2560c28be7f253f35b9e8b5cc79063045005b73b194dceed8
  Stored in directory: /tmp/pip-ephem-wheel-cache-dzodiv8g/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.invariant_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [6]:
from torch.optim import SGD

erm = ERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    batch_size=32,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4),
    device=device,
    verbose=True
)

In [7]:
erm.train()

Epoch 4: 100%|██████████| 938/938 [00:03<00:00, 264.66batch/s, accuracy=100.0%, loss=0.058]   


In [8]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 98.81796690307328
Group (0, 1) Test Accuracy: 0.0
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 0.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 99.73333333333333
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 1.2594458438287153
Group (3, 3) Test Accuracy: 100.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.5037783375314862
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 98.81796690307328,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 99.73333333333333,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 1.2594458438287153,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.5037783375314862,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [9]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [10]:
evaluator.average_accuracy

99.28700461959575

In [11]:
evaluator.evaluate_spurious_task()

99.46

In [32]:
import random 
from torch.utils.data import Subset

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

group_balanced_set = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.2,
    classes=classes,
    train=True
)
group_balanced_set.initialize()
group_balanced_set = Subset(group_balanced_set, [i for key in group_balanced_set.group_partition.keys() for i in group_balanced_set.group_partition[key]])

In [38]:
from spuco.finetune import DFR 
from torch.optim import SGD

dfr = DFR(
    group_balanced_dataset=group_balanced_set,
    model=model,
    num_epochs=100,
    lr=1e-2,
    momentum=0.9,
    verbose=True,
    device=device
)

In [39]:
dfr.train()

Epoch 99: 100%|██████████| 938/938 [00:02<00:00, 365.83batch/s, accuracy=34.375%, loss=1.53] 


In [40]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 65.01182033096927
Group (0, 1) Test Accuracy: 47.990543735224584
Group (0, 2) Test Accuracy: 52.009456264775416
Group (0, 3) Test Accuracy: 54.60992907801418
Group (0, 4) Test Accuracy: 9.456264775413711
Group (1, 0) Test Accuracy: 52.07823960880196
Group (1, 1) Test Accuracy: 71.39364303178485
Group (1, 2) Test Accuracy: 35.049019607843135
Group (1, 3) Test Accuracy: 20.58823529411765
Group (1, 4) Test Accuracy: 0.9803921568627451
Group (2, 0) Test Accuracy: 55.733333333333334
Group (2, 1) Test Accuracy: 48.53333333333333
Group (2, 2) Test Accuracy: 40.53333333333333
Group (2, 3) Test Accuracy: 24.8
Group (2, 4) Test Accuracy: 48.663101604278076
Group (3, 0) Test Accuracy: 46.733668341708544
Group (3, 1) Test Accuracy: 36.02015113350126
Group (3, 2) Test Accuracy: 39.7984886649874
Group (3, 3) Test Accuracy: 21.15869017632242
Group (3, 4) Test Accuracy: 8.816120906801007
Group (4, 0) Test Accuracy: 23.67758186397985
Group (4, 1) Test Accuracy: 55.1637279596

{(0, 0): 65.01182033096927,
 (0, 1): 47.990543735224584,
 (0, 2): 52.009456264775416,
 (0, 3): 54.60992907801418,
 (0, 4): 9.456264775413711,
 (1, 0): 52.07823960880196,
 (1, 1): 71.39364303178485,
 (1, 2): 35.049019607843135,
 (1, 3): 20.58823529411765,
 (1, 4): 0.9803921568627451,
 (2, 0): 55.733333333333334,
 (2, 1): 48.53333333333333,
 (2, 2): 40.53333333333333,
 (2, 3): 24.8,
 (2, 4): 48.663101604278076,
 (3, 0): 46.733668341708544,
 (3, 1): 36.02015113350126,
 (3, 2): 39.7984886649874,
 (3, 3): 21.15869017632242,
 (3, 4): 8.816120906801007,
 (4, 0): 23.67758186397985,
 (4, 1): 55.16372795969773,
 (4, 2): 38.79093198992443,
 (4, 3): 53.03030303030303,
 (4, 4): 0.0}

In [36]:
evaluator.worst_group_accuracy

((0, 3), 0.0)